<a href="https://colab.research.google.com/github/Maryam-Zubair/Data-Science/blob/main/9_01_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

data = pd.read_csv('adult.csv')

Int his lab assignment, you have two tasks:
1. Create a dataset from the `adult.csv` where the new dataset will have columns with random number of missing values. The ratio of missing values should randomly change from 5% to 10%. The numerical columns will have `np.NaN`, and categorical ones will have `?` as missing values.
For example,the first column, age, is numerical. It may have a randomly chosen 6% of the age values as missing.
At this stage, take they type of each coumn as the datatype. For example, `marital_status` looks like a numerical column at the beginning. Tka it as a numerical column and replace a random percentage of values with `np.NaN`.
2. Create a pipeline with transformers and a classifier `SVC(kernel='rbf', C=1, gamma='scale')`. When creating the transformers, make sure that each column is correctly classified as numerical or categorical. For example, `marital_status` may have a `float` data type, but in reality it is a categorical variable. For that reason, you need to update the data types for all categorical variables.Report the accuracy for your model.
3. Repeat the second step for the original dataset without any missing values. Compatre your result with that from the second step.

In [ ]:
def insert_missing_values(df):
    for column in df.columns:
        if column != 'income':
            is_numeric = pd.api.types.is_numeric_dtype(df[column])
            mask = df.sample(frac=np.random.uniform(0.05, 0.1)).index
            df.loc[mask, column] = np.nan if is_numeric else '?'
    return df

In [ ]:
new_data = insert_missing_values(data.copy())

In [ ]:
categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
numerical_cols = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

In [ ]:
for col in categorical_cols:
    new_data[col] = new_data[col].astype('category')

In [ ]:
# Define the preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

In [ ]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC(kernel='rbf', C=1, gamma='scale'))])

In [ ]:
# Apply the transformations
X = new_data.drop('income', axis=1)
y = new_data['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.571


Now We Do it with original data without putting missing values

In [ ]:
# Now with original unedited data
X = data.drop('income', axis=1)
y = data['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
print("model score with original data: %.3f" % clf.score(X_test, y_test))

model score with original data: 0.573
